In [49]:
%run params.py

import pandas as pd
from plotting_functions import *
import numpy as np


lobref = pd.read_csv(os.path.join(DATAPATH, "ref", "lobstr_v3.0.2_hg19_ref_nochr.bed.gz"), usecols=range(4),
                     names=["chrom","start","end","period"], sep="\t")

data = LoadMLData(os.path.join(DATAPATH, "autosomal_estimates/perlocus", "autosomal_estimates_ml_filtered_v2.bed.gz"))
sdata = pd.read_csv(os.path.join(DATAPATH, "autosomal_estimates/perlocus", "autosomal_stutter_ml_v2.bed.gz"), names=["chrom","start","end","up","down","p"], sep="\t")
zdata = pd.read_csv(os.path.join(DATAPATH, "constraint", "autosomal_perlocus_estimates.bed"))
smmzdata = pd.read_csv(os.path.join(DATAPATH, "constraint", "autosomal_perlocus_estimates_smm.bed"))

In [50]:
data["chrom"] = data["chrom"].apply(str)
data["start"] = data["start"].apply(int)
data["end"] = data["end"].apply(int)
sdata["chrom"] = sdata["chrom"].apply(str)
sdata["start"] = sdata["start"].apply(int)
sdata["end"] = sdata["end"].apply(int)
zdata["chrom"] = zdata["chrom"].apply(str)
zdata["start"] = zdata["start"].apply(int)
zdata["end"] = zdata["end"].apply(int)
smmzdata["chrom"] = smmzdata["chrom"].apply(str)
smmzdata["start"] = smmzdata["start"].apply(int)
smmzdata["end"] = smmzdata["end"].apply(int)
lobref["chrom"] = lobref["chrom"].apply(str)
lobref["start"] = lobref["start"].apply(int)
lobref["end"] = lobref["end"].apply(int)

In [51]:
suppdata = pd.merge(data, sdata, on=["chrom","start","end"])
suppdata = pd.merge(suppdata, lobref, on=["chrom","start","end"])
suppdata = pd.merge(suppdata, zdata, on=["chrom","start","end"], how="outer")
suppdata = pd.merge(suppdata, smmzdata[["chrom","start","end","smm_mu","zscore_1_smm","filter1_smm"]],
                    on=["chrom","start","end"], how="outer").drop_duplicates()

In [52]:
# Small changes
suppdata["#chrom"] = suppdata["chrom"]
suppdata["numsamples_ml"] = suppdata["numsamples_ml"].apply(int)

In [53]:
# Add column indicating if used in training
traindata = pd.read_csv(os.path.join(DATAPATH, "constraint", "autosomal_perlocus_train_intergenic.bed.gz"), usecols=range(3),
                       sep="\t").drop_duplicates()
traindata["train"] = 1
traindata["chrom"] = traindata["chrom"].apply(str)
trainloci = set()
for i in range(traindata.shape[0]): trainloci.add((traindata.chrom.values[i], traindata.start.values[i]))
suppdata["train_2"] = suppdata.apply(lambda x: (x.chrom, x.start) in trainloci, 1)
suppdata["train_1"] = suppdata.apply(lambda x: x.train_2 and not x.filter1, 1)

In [54]:
# Adjust SMM rates by missing factor of 2 (was missing in original run)
suppdata["smm_mu"] = suppdata["smm_mu"] - np.log10(2)
# Drop duplicate rows, usually occurs because more than 1 motif for that repeat
suppdata = suppdata.drop_duplicates(subset=["chrom","start","end"])

In [55]:
columns = ["#chrom","start","end","est_logmu_ml","est_beta_eff_ml","est_beta_ml", \
          "est_pgeom_ml", "stderr_ml", "numsamples_ml", \
          "up", "down", "p", \
          "period", "motif","uninterrupted_length", \
          "pred_mu_1", "pred_mu_se_1", "zscore_1", \
          "pred_mu_2", "pred_mu_se_2", "zscore_2", "filter1","train_1", "train_2", \
          "smm_mu","zscore_1_smm","filter1_smm"]
suppdata[columns].to_csv(os.path.join(DATAPATH, "suppdata", "Gymrek_etal_SupplementalData1_v2.bed"), sep="\t", index=False)

In [56]:
suppdata.shape

(1250930, 35)